## 데이터마이닝 프로젝트

In [172]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

import warnings
warnings.filterwarnings('ignore')


In [173]:
# 데이터 로드

FP_PATH    = "floating_population.csv"   # 유동인구
SALES_PATH = "sales.csv"                 # 매출
STORE_PATH = "store.csv"                 # 점포


In [ ]:
def read_korean_csv(path: str) -> pd.DataFrame:
    """Try common Korean encodings."""
    for enc in ("utf-8", "cp949", "euc-kr"):
        try:
            return pd.read_csv(path, encoding=enc)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path)

fp    = read_korean_csv(FP_PATH)
fp = fp[fp["기준_년분기_코드"] // 10 == 2024]
sales = read_korean_csv(SALES_PATH)
store = read_korean_csv(STORE_PATH)

print(fp.shape, sales.shape, store.shape)


In [ ]:
f = lambda x: display(x["기준_년분기_코드"].unique())
f(fp)
f(sales)
f(store)

In [ ]:
def get_columns(df):
    display(df.columns)
get_columns(fp)
get_columns(sales)
get_columns(store)

In [ ]:
#데이터 전처리
def make_numeric(df, drop_cols):
    return [c for c in df.columns if c not in drop_cols and df[c].dtype != 'object']

sales_num = make_numeric(sales, ["행정동_코드", "기준_년분기_코드"])
store_num = make_numeric(store, ["행정동_코드", "기준_년분기_코드"])

sales_agg = (sales
             .groupby(["기준_년분기_코드", "행정동_코드"])[sales_num]
             .sum()
             .reset_index())

store_agg = (store
             .groupby(["기준_년분기_코드", "행정동_코드"])[store_num]
             .median() # 0 값이 많아 median과 차이가 있는데 어떤 걸로 할지?
             .reset_index())

# merged 건들면 안됨!
merged = (fp
          .merge(sales_agg, on=["기준_년분기_코드", "행정동_코드"], how="left")
          .merge(store_agg, on=["기준_년분기_코드", "행정동_코드"], how="left"))
merged = merged.dropna(axis=0)
print("Merged:", merged.shape)


In [ ]:
merged.isna().sum().sum()

In [ ]:
# 표준화 후 PCA 분석
feature_cols = [c for c in merged.columns
                if c not in ("행정동_코드", "기준_년분기_코드") and merged[c].dtype != 'object']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(merged[feature_cols])

pca = PCA(n_components=0.95, random_state=42, svd_solver='full')
X_pca = pca.fit_transform(X_scaled)

print(f"PCA → {X_pca.shape[1]} components (cum var {pca.explained_variance_ratio_.sum():.2%})")


In [180]:
# print(f"PCA → {X_pca.shape[1]} components (each var \n{pca.explained_variance_ratio_})")

-> 8개 주성분으로 원본 데이터의 분산 95.05% 설명 가능

In [181]:
# loadings = pd.DataFrame(
#     pca.components_.T,                         # (변수 개수 × 주성분 개수) 행렬
#     index=feature_cols,                        # 행 = 원본 변수 이름
#     columns=[f"PC{i+1}" for i in range(pca.n_components_)]
# )

In [182]:
# len(merged.columns)

In [183]:
# for pc in loadings.columns:          # 'PC1' … 'PC7'
#     top = (loadings[pc]
#            .sort_values(ascending=False, key=abs)
#            .head(30))                # 상위 8개 변수
#     print(f"\n{pc} 상위 기여 변수:")
#     print(top)

In [184]:
# # kmeans Clustering 사용하기 위해 k 설정하는 과정(지표 비교)

# def eval_k(X, k):
#     km = KMeans(n_clusters=k, random_state=42, n_init=10).fit(X)
#     labels = km.labels_
#     return {
#         "k": k,
#         "inertia": km.inertia_,
#         "silhouette": silhouette_score(X, labels, sample_size=min(5000, len(X)), random_state=42),
#         "calinski": calinski_harabasz_score(X, labels),
#         "davies": davies_bouldin_score(X, labels)
#     }

# scores = pd.DataFrame([eval_k(X_pca, k) for k in range(2, 11)])
# display(scores.set_index("k").round(2))

# best_k = scores.loc[scores['silhouette'].idxmax(), 'k']
# print("Best k (silhouette):", best_k)


In [185]:
# from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt

# ks, inertias = [], []
# for k in range(2, 11):
#     km = KMeans(n_clusters=k, random_state=42, n_init=10).fit(X_pca)
#     ks.append(k)
#     inertias.append(km.inertia_)

# plt.plot(ks, inertias, 'o-')
# plt.title("Elbow Method")
# plt.xlabel("k"); plt.ylabel("Inertia")
# plt.show()

In [186]:
### silhouette score는 k=2에서 제일 높았지만, 2개의 클러스터는 적다고 느껴지기도 하고 엘보우 메소드를 참고해서 k=4로 해봤습니다!

In [187]:
# # kmeans 클러스터링 적용
# kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
# merged['cluster'] = kmeans.fit_predict(X_pca)
# merged['cluster'].value_counts().sort_index()


In [188]:
# # 시각화
# plt.figure(figsize=(5,4))
# plt.scatter(X_pca[:,0], X_pca[:,1], c=merged['cluster'], s=8, cmap='tab10')
# plt.title('Clusters on PC1 vs PC2')
# plt.show()  #PCA 1·2 축 산점도

# plt.figure(figsize=(4,3))
# merged['cluster'].value_counts().sort_index().plot(kind='bar')
# plt.title('Cluster sizes')
# plt.show() #cluster 별 크기

# pc_df = pd.DataFrame(X_pca, columns=[f"PC{i+1}" for i in range(X_pca.shape[1])])
# pc_df["cluster"] = merged["cluster"]
# mean_matrix = pc_df.groupby("cluster").mean()

# plt.figure(figsize=(6,3))
# plt.imshow(mean_matrix, aspect="auto"); plt.colorbar()
# plt.yticks(range(len(mean_matrix)), mean_matrix.index)
# plt.title("Cluster Mean by PC"); plt.xlabel("PC"); plt.ylabel("Cluster")
# plt.tight_layout(); plt.show() #군집별 PC 평균 Heatmap



In [189]:
# from sklearn.manifold import TSNE

# for i in [10, 30, 50, 70]:
#     tsne_model = TSNE(
#         n_components=2,      # 축소할 차원 (보통 2 또는 3)
#         perplexity=i,     # 가장 중요한 파라미터 중 하나. 데이터 포인트 주변의 로컬 이웃 수와 관련. (5 ~ 50 사이 값 권장)
#         learning_rate='auto',# 최적화 과정에서의 학습률. 'auto' (scikit-learn 1.2 이상) 또는 10 ~ 1000 사이 값.
#                             # 이전 버전에서는 learning_rate=200.0 이 기본값이었음.
#         n_iter=1000,         # 최적화를 위한 반복 횟수 (최소 250, 보통 1000 이상 권장)
#         init='pca',          # 초기 임베딩 방법 ('random' 또는 'pca'). 'pca'가 더 안정적이고 좋은 결과를 주는 경향이 있음.
#         random_state=42      # 결과 재현을 위한 시드 값
#     )

#     # t-SNE 적용 (원본 X 또는 X_scaled 사용)
#     tsne_results = tsne_model.fit_transform(X_scaled) # 또는 X_scaled

#     # 2D 시각화
#     plt.figure(figsize=(10, 7))
#     scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=merged["cluster"], cmap='viridis', alpha=0.7)


#     plt.title('t-SNE visualization of data')
#     plt.xlabel('t-SNE Component 1')
#     plt.ylabel('t-SNE Component 2')
#     plt.grid(True)
#     plt.show()

In [190]:
# #PC 평균
# display(
#     mean_matrix.round(2)
#                .style.background_gradient(cmap="Blues", axis=None)
#                .set_caption("클러스터별 주성분 평균")
# )
# #원본 변수 — 변별력 상위 8개
# cluster_avg = merged.groupby("cluster")[feature_cols].mean()
# top_vars = cluster_avg.var().sort_values(ascending=False).head(8).index

# display(
#     cluster_avg[top_vars].T.round(1)
#                .style.background_gradient(cmap="YlOrRd", axis=1)
#                .set_caption("클러스터별 평균 — 변별력 상위 8개 변수")
# )

In [191]:
# merged에서 일부 열만 추출한 걸로 클러스터링 해보기
# 
new_merged = merged[['점포_수',"행정동_코드","당월_매출_건수", "당월_매출_금액",'시간대_06~11_매출_금액', '시간대_11~14_매출_금액','시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액']]
new_merged['여성_비율'] = merged['여성_매출_건수'] / (merged['남성_매출_건수']+merged['여성_매출_건수'])

new_merged['20대_매출_금액_비율'] = merged['연령대_20_매출_금액'] / merged['당월_매출_금액']
new_merged['20대_유동인구_비율'] = merged['연령대_20_유동인구_수'] / (merged['연령대_50_유동인구_수']+merged['연령대_20_유동인구_수']+merged['연령대_30_유동인구_수']+merged['연령대_40_유동인구_수']+merged['연령대_60_이상_유동인구_수'])



In [ ]:
seoul_gu_code_map = {
    '11110': '종로구',
    '11140': '중구',
    '11170': '용산구',
    '11200': '성동구',
    '11215': '광진구',
    '11230': '동대문구',
    '11260': '중랑구',
    '11290': '성북구',
    '11305': '강북구',
    '11320': '도봉구',
    '11350': '노원구',
    '11380': '은평구',
    '11410': '서대문구',
    '11440': '마포구',
    '11470': '양천구',
    '11500': '강서구',
    '11530': '구로구',
    '11545': '금천구',
    '11560': '영등포구',
    '11590': '동작구',
    '11620': '관악구',
    '11650': '서초구',
    '11680': '강남구',
    '11710': '송파구',
    '11740': '강동구'
}

import pandas as pd

# 예시 DataFrame
# merged = pd.read_csv("your_file.csv")  # 실제 사용할 때

# 행정동_코드를 문자열로 변환
merged['행정동_코드'] = merged['행정동_코드'].astype(str)

# 시군구 코드 추출 (앞 5자리)
merged['시군구코드'] = merged['행정동_코드'].str[:5]

# 자치구명 매핑
merged['자치구'] = merged['시군구코드'].map(seoul_gu_code_map)

# 결과 확인
print(merged[['행정동_코드', '시군구코드', '자치구']])

In [ ]:

# 새로운 데이터셋에 대한 PCA 수행
scaler = StandardScaler()
X_scaled = scaler.fit_transform(new_merged)

# 2개짜리 pca
pca2 = PCA(n_components=2)
X_pca2 = pca2.fit_transform(X_scaled)

pca = PCA(n_components=3)  # 2개의 주성분만 사용
X_pca = pca.fit_transform(X_scaled)

print(pca.components_)
print(pca.n_components_)


In [ ]:
# 클러스터링 및 2차원 시각화

kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
new_merged['cluster'] = kmeans.fit_predict(X_pca)

# 2차원 시각화
plt.figure(figsize=(10, 8))
plt.scatter(X_pca[:,0], X_pca[:,1], c=new_merged['cluster'], cmap='viridis', alpha=0.3)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()


In [ ]:
# 클러스터링 적용
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
new_merged['cluster'] = kmeans.fit_predict(X_pca)

# 3차원 시각화
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_pca[:,0], X_pca[:,1], X_pca[:,2], c=new_merged['cluster'], cmap='viridis', alpha=0.3)

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.show()


In [ ]:
# 클러스터별 통계 분석
cluster_stats = new_merged.groupby('cluster').agg({
    '점포_수': 'mean',
    '당월_매출_건수': 'mean',
    '당월_매출_금액': 'mean',
    '시간대_06~11_매출_금액': 'mean',
    '시간대_11~14_매출_금액': 'mean',
    '시간대_14~17_매출_금액': 'mean',
    '시간대_17~21_매출_금액': 'mean',
    '시간대_21~24_매출_금액': 'mean',
    '여성_비율': 'mean',
    '20대_매출_금액_비율': 'mean',
    '20대_유동인구_비율': 'mean'
}).round(2)

# 각 클러스터별 샘플 데이터 확인 (각 클러스터에서 3개씩)
cluster_samples = {}
for cluster in new_merged['cluster'].unique():
    cluster_samples[cluster] = new_merged[new_merged['cluster'] == cluster].sample(3)

# 결과 출력
print("=== 클러스터별 평균 통계 ===")
display(cluster_stats)

print("\n=== 클러스터별 샘플 데이터 ===")
for cluster, samples in cluster_samples.items():
    print(f"\n클러스터 {cluster}의 샘플:")
    display(samples)

In [ ]:

# DBSCAN으로 클러스터링 해보기
from sklearn.cluster import DBSCAN


# DBSCAN 적용
dbscan = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
new_merged['cluster'] = dbscan.fit_predict(X_pca)

# 3d 시각화

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_pca[:,0], X_pca[:,1], X_pca[:,2], c=new_merged['cluster'], cmap='viridis', alpha=0.1)

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.show()


In [198]:
selected_df = pd.DataFrame()
selected_df["총_유동인구_수"] = merged["총_유동인구_수"]
selected_df["청년층_유동인구_비율"] = (merged["연령대_20_유동인구_수"] + merged["연령대_30_유동인구_수"])/merged["총_유동인구_수"]
selected_df["청년_매출_비율"] = (merged["연령대_20_매출_건수"] + merged["연령대_30_매출_건수"]) / merged["당월_매출_건수"]
selected_df["주말_매출_비율"] =  merged["주중_매출_건수"]/ merged["주말_매출_건수"]




# # 4.1 이상치 처리
# def remove_outliers(df, columns):
#     df_clean = df.copy()
#     for col in columns:
#         Q1 = df[col].quantile(0.25)
#         Q3 = df[col].quantile(0.75)
#         IQR = Q3 - Q1
#         lower_bound = Q1 - 1.5 * IQR
#         upper_bound = Q3 + 1.5 * IQR
#         df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
#     return df_clean

# # 이상치가 많이 발생할 수 있는 변수들 선택
# outlier_columns = ['총_유동인구_수', '청년층_유동인구_비율', '청년_매출_비율', '주말_매출_비율']
# selected_df = remove_outliers(selected_df, outlier_columns)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(selected_df)

pca2 = PCA(n_components=2)
X_pca2 = pca2.fit_transform(X_scaled)

pca3 = PCA(n_components=3)
X_pca3 = pca3.fit_transform(X_scaled)


In [199]:
# 클러스터 로직
selected_df['cluster'] = kmeans.fit_predict(X_pca3)

In [ ]:
# 3d 시각화
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_pca3[:,0], X_pca3[:,1], X_pca3[:,2], c=selected_df['cluster'], cmap='viridis', alpha=0.3)

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.show()




In [ ]:
# 클러스터별 통계 분석
cluster_stats = selected_df.groupby('cluster').agg({
    "총_유동인구_수": 'mean',
    "청년층_유동인구_비율": 'mean',
    "청년_매출_비율": 'mean',
    "주말_매출_비율": 'mean',
}).round(2)

# 각 클러스터별 샘플 데이터 확인 (각 클러스터에서 3개씩)
cluster_samples = {}
for cluster in selected_df['cluster'].unique():
    cluster_samples[cluster] = selected_df[selected_df['cluster'] == cluster].sample(3)

# 결과 출력
print("=== 클러스터별 평균 통계 ===")
display(cluster_stats)

print("\n=== 클러스터별 샘플 데이터 ===")
for cluster, samples in cluster_samples.items():
    print(f"\n클러스터 {cluster}의 샘플:")
    display(samples)

#  실루엣 계수
from sklearn.metrics import silhouette_score

silhouette_scores = []

for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_pca3)
    labels = kmeans.labels_
    score = silhouette_score(X_pca3, labels)
    silhouette_scores.append(score)

plt.figure(figsize=(10, 5))
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.show()

# 최적의 클러스터 수 찾기
best_k = silhouette_scores.index(max(silhouette_scores)) + 2
print(f"최적의 클러스터 수: {best_k}")




CURSOR 가이드라인

In [202]:
# 핫플레이스 특성을 더 잘 반영하는 변수들로 재구성
selected_df = pd.DataFrame()
selected_df["총_유동인구_수"] = merged["총_유동인구_수"]
selected_df["당월_매출_금액"] = merged["당월_매출_금액"]
selected_df["야간유동인구비율"] = (merged["시간대_17_21_유동인구_수"] 
                            + merged["시간대_21_24_유동인구_수"] 
                            + merged["시간대_00_06_유동인구_수"]) / merged["총_유동인구_수"]
selected_df["청년층_유동인구_비율"] = (merged["연령대_20_유동인구_수"] + merged["연령대_30_유동인구_수"])/merged["총_유동인구_수"]
selected_df["점포당_월매출액"] = merged["당월_매출_금액"] / merged["유사_업종_점포_수"]
selected_df["청년_매출_비율"] = (merged["연령대_20_매출_건수"] + merged["연령대_30_매출_건수"]) / merged["당월_매출_건수"]
selected_df["주말_매출_비율"] = merged["주중_매출_건수"] / merged["주말_매출_건수"]
selected_df["객단가"] = merged["당월_매출_금액"] / merged["당월_매출_건수"]
selected_df["개업_율"] = merged["개업_율"]
selected_df["프랜차이즈_점포_비율"] = merged["프랜차이즈_점포_수"] / merged["유사_업종_점포_수"]

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(selected_df)

pca2 = PCA(n_components=2)
X_pca2 = pca2.fit_transform(X_scaled)

pca3 = PCA(n_components=3)
X_pca3 = pca3.fit_transform(X_scaled)

kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
kmeans3d = KMeans(n_clusters=5, random_state=42, n_init=10)
selected_df['cluster'] = kmeans.fit_predict(X_pca2)
selected_df['cluster_3'] = kmeans3d.fit_predict(X_pca3)

# 클러스터 시각화
plt.figure(figsize=(10, 8))
plt.scatter(X_pca2[:,0], X_pca2[:,1], c=selected_df['cluster'], cmap='viridis', alpha=0.3)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

# 클러스터 3d 시각화
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_pca3[:,0], X_pca3[:,1], X_pca3[:,2], c=selected_df['cluster_3'], cmap='viridis', alpha=0.3)

ax.set_xlabel('PC1')





In [ ]:
print(merged['행정동_코드'].astype(str).str[:3])

In [ ]:
# 행정동 코드에서 구 정보 추출
merged['구'] = merged['행정동_코드'].astype(str).str[:3]

# 구별 특성 분석
구별_특성 = merged.groupby('구').agg({
    '당월_매출_금액': 'mean',
    '총_유동인구_수': 'mean',
    '개업_율': 'mean',
    '폐업_률': 'mean',
    '프랜차이즈_점포_수': 'mean',
    '점포_수': 'mean',
    '연령대_20_유동인구_수': 'mean',
    '시간대_17_21_유동인구_수': 'mean',
    '시간대_21_24_유동인구_수': 'mean'
}).round(2)

# 구별 성장성 지수 계산
구별_특성['성장성_지수'] = (
    구별_특성['개업_율'] - 구별_특성['폐업_률'] + 
    구별_특성['당월_매출_금액'] / 구별_특성['당월_매출_금액'].mean() +
    구별_특성['총_유동인구_수'] / 구별_특성['총_유동인구_수'].mean()
) / 3

# 구별 청년층 비율 계산
구별_특성['청년층_비율'] = 구별_특성['연령대_20_유동인구_수'] / 구별_특성['총_유동인구_수']

# 구별 야간 유동인구 비율 계산
구별_특성['야간_유동인구_비율'] = (
    구별_특성['시간대_17_21_유동인구_수'] + 
    구별_특성['시간대_21_24_유동인구_수']
) / 구별_특성['총_유동인구_수']

# 결과 출력
print("\n=== 구별 주요 통계 ===")
display(구별_특성.sort_values('성장성_지수', ascending=False))

# 시각화
plt.figure(figsize=(12, 6))
구별_특성['성장성_지수'].sort_values().plot(kind='bar')
plt.title('구별 성장성 지수')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 구 클러스터 만들기
scaler = StandardScaler()
X_scaled = scaler.fit_transform(구별_특성)

pca2 = PCA(n_components=2)
X_pca2 = pca2.fit_transform(X_scaled)

pca3 = PCA(n_components=3)
X_pca3 = pca3.fit_transform(X_scaled)

kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
구별_특성['cluster'] = kmeans.fit_predict(X_pca2)

# 구별특성 클러스터 시각화
plt.figure(figsize=(10, 8))
plt.scatter(X_pca2[:,0], X_pca2[:,1], c=구별_특성['cluster'], cmap='viridis', alpha=0.3)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()



# 구별 클러스터 분포 확인
구별_클러스터_분포 = pd.crosstab(merged['구'], 구별_특성['cluster'])
print("\n=== 구별 클러스터 분포 ===")
display(구별_클러스터_분포)

# 구별 핫플레이스 잠재력 지수 계산
구별_특성['핫플레이스_잠재력_지수'] = (
    구별_특성['당월_매출_금액'] / 구별_특성['당월_매출_금액'].mean() * 0.3 +
    구별_특성['총_유동인구_수'] / 구별_특성['총_유동인구_수'].mean() * 0.2 +
    구별_특성['개업_율'] * 0.2 +
    구별_특성['청년층_비율'] * 0.2 +
    구별_특성['야간_유동인구_비율'] * 0.1
)

print("\n=== 구별 핫플레이스 잠재력 지수 ===")
display(구별_특성.sort_values('핫플레이스_잠재력_지수', ascending=False)[['핫플레이스_잠재력_지수', '성장성_지수', '청년층_비율', '야간_유동인구_비율']])

In [ ]:
# import seaborn as sns

# # 구별특성 클러스터 시각화
# plt.figure(figsize=(12, 6))
# sns.scatterplot(x='핫플레이스_잠재력_지수', y='성장성_지수', hue='cluster', data=구별_특성, palette='viridis')
# plt.title('구별 핫플레이스 잠재력 지수와 성장성 지수 시각화')
# plt.xlabel('핫플레이스 잠재력 지수')
# plt.ylabel('성장성 지수')
# plt.show()

# print(merged['구'])

# # merged 클러스터 시각화
# plt.figure(figsize=(12, 6))
# sns.scatterplot(x='핫플레이스_잠재력_지수', y='성장성_지수', hue='cluster', data=merged, palette='viridis')
# plt.title('구별 핫플레이스 잠재력 지수와 성장성 지수 시각화')
# plt.xlabel('핫플레이스 잠재력 지수')
# plt.ylabel('성장성 지수')
# plt.show()


In [ ]:
# 핫플레이스 잠재력 지수 계산
merged['핫플레이스_잠재력_지수'] = (
    merged['당월_매출_금액'] / merged['당월_매출_금액'].mean() * 0.3 +
    merged['총_유동인구_수'] / merged['총_유동인구_수'].mean() * 0.2 +
    merged['개업_율'] * 0.2 +
    (merged['연령대_20_유동인구_수'] + merged['연령대_30_유동인구_수']) / merged['총_유동인구_수'] * 0.2 +
    (merged['시간대_17_21_유동인구_수'] + merged['시간대_21_24_유동인구_수']) / merged['총_유동인구_수'] * 0.1
)

# 잠재력 지수 상위 지역 분석
잠재력_상위지역 = merged.nlargest(10, '핫플레이스_잠재력_지수')
print("\n=== 핫플레이스 잠재력 상위 10개 지역 ===")
display(잠재력_상위지역[['행정동_코드', '핫플레이스_잠재력_지수', '당월_매출_금액', '총_유동인구_수', '개업_율']])

In [ ]:
import seaborn as sns

# 히트맵으로 지역별 특성 시각화
plt.figure(figsize=(12, 8))
sns.heatmap(구별_특성, annot=True, cmap='YlOrRd', fmt='.2f')
plt.title('구별 특성 히트맵')
plt.show()

# 산점도로 핵심 변수 간 관계 시각화
plt.figure(figsize=(10, 6))
plt.scatter(merged['총_유동인구_수'], merged['당월_매출_금액'], 
           c=merged['핫플레이스_잠재력_지수'], cmap='viridis', alpha=0.5)
plt.colorbar(label='핫플레이스 잠재력 지수')
plt.xlabel('총 유동인구 수')
plt.ylabel('당월 매출 금액')
plt.title('유동인구와 매출액의 관계')
plt.show()

In [ ]:
# 필요한 라이브러리 import
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Nanum Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 스타일 설정
# plt.style.use('seaborn')
sns.set_palette("husl")

# 이상치 제거 함수
def remove_outliers(df, columns):
    df_clean = df.copy()
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    return df_clean

# 이상치가 많이 발생할 수 있는 변수들 선택
outlier_columns = ['총_유동인구_수', '청년층_유동인구_비율', '청년_매출_비율', '주말_매출_비율']

# 자치구 정보 추가
selected_df["자치구"] = merged["자치구"]

# 이상치 제거 전에 자치구 정보 저장
district_info = selected_df['자치구'].copy()

# 이상치 제거
selected_df = remove_outliers(selected_df, outlier_columns)

# 이상치 제거 후 남은 데이터에 대한 자치구 정보 업데이트
district_info = district_info[selected_df.index]

# 스케일링 및 PCA 적용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(selected_df[outlier_columns])

# 2차원 PCA
pca2 = PCA(n_components=2)
X_pca2 = pca2.fit_transform(X_scaled)

# 3차원 PCA
pca3 = PCA(n_components=3)
X_pca3 = pca3.fit_transform(X_scaled)

# 자치구별 고유한 색상 매핑 (더 선명한 색상 사용)
unique_districts = district_info.unique()
colors = plt.cm.tab20(np.linspace(0, 1, len(unique_districts)))  # tab20 컬러맵 사용
district_colors = dict(zip(unique_districts, colors))

# 시각화를 위한 네 가지 방법 구현
fig = plt.figure(figsize=(20, 15))
gs = fig.add_gridspec(2, 2)

# 1. 밀도 기반 시각화 (KDE) - 개선된 버전
ax1 = fig.add_subplot(gs[0, 0])
for district in unique_districts:
    mask = district_info == district
    points = X_pca2[mask]
    
    if len(points) > 1:
        # KDE 계산
        kde = gaussian_kde(points.T)
        
        # 전체 데이터의 범위를 기준으로 그리드 생성
        x_min, x_max = X_pca2[:, 0].min(), X_pca2[:, 0].max()
        y_min, y_max = X_pca2[:, 1].min(), X_pca2[:, 1].max()
        
        # 여백 추가
        x_margin = (x_max - x_min) * 0.1
        y_margin = (y_max - y_min) * 0.1
        
        x_grid = np.linspace(x_min - x_margin, x_max + x_margin, 200)
        y_grid = np.linspace(y_min - y_margin, y_max + y_margin, 200)
        X, Y = np.meshgrid(x_grid, y_grid)
        positions = np.vstack([X.ravel(), Y.ravel()])
        
        # 밀도 계산
        Z = np.reshape(kde(positions).T, X.shape)
        
        # 밀도 플롯 - 개선된 버전
        ax1.contour(X, Y, Z, colors=[district_colors[district]], alpha=0.7, linewidths=1.5)
        ax1.contourf(X, Y, Z, colors=[district_colors[district]], alpha=0.2)
        
        # 중심점 표시 - 크기와 스타일 개선
        center = points.mean(axis=0)
        ax1.scatter(center[0], center[1], 
                   c=[district_colors[district]], 
                   label=district, 
                   s=200,  # 크기 증가
                   edgecolor='black',
                   linewidth=2,
                   zorder=3)

# 그래프 스타일 개선
ax1.set_xlabel('PC1', fontsize=12)
ax1.set_ylabel('PC2', fontsize=12)
ax1.set_title('자치구별 밀도 기반 시각화', fontsize=15, pad=20)
ax1.grid(True, linestyle='--', alpha=0.3)
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)

# 2. 버블 차트 - 개선된 버전
ax2 = fig.add_subplot(gs[0, 1])
for district in unique_districts:
    mask = district_info == district
    points = X_pca2[mask]
    
    # 데이터 포인트 수에 따른 크기 계산 - 크기 조정
    size = 300 + (len(points) / len(district_info)) * 3000
    
    # 중심점 계산
    center = points.mean(axis=0)
    
    # 버블 플롯 - 스타일 개선
    ax2.scatter(center[0], center[1], 
                c=[district_colors[district]], 
                label=district,
                s=size,
                alpha=0.7,
                edgecolor='black',
                linewidth=2)
    
    # 자치구 이름 표시 - 위치와 스타일 개선
    ax2.annotate(district, 
                (center[0], center[1]),
                xytext=(10, 10),
                textcoords='offset points',
                fontsize=10,
                bbox=dict(facecolor='white', 
                         edgecolor='none',
                         alpha=0.7,
                         pad=2))

# 그래프 스타일 개선
ax2.set_xlabel('PC1', fontsize=12)
ax2.set_ylabel('PC2', fontsize=12)
ax2.set_title('자치구별 버블 차트', fontsize=15, pad=20)
ax2.grid(True, linestyle='--', alpha=0.3)

# 3. 3차원 PCA 시각화
ax3 = fig.add_subplot(gs[1, 0], projection='3d')
for district in unique_districts:
    mask = district_info == district
    points = X_pca3[mask]
    
    # 3D 산점도
    ax3.scatter(points[:, 0], points[:, 1], points[:, 2],
                c=[district_colors[district]],
                label=district,
                alpha=0.6,
                s=50,
                edgecolor='black',
                linewidth=0.5)

ax3.set_xlabel('PC1', fontsize=12)
ax3.set_ylabel('PC2', fontsize=12)
ax3.set_zlabel('PC3', fontsize=12)
ax3.set_title('자치구별 3차원 PCA 시각화', fontsize=15, pad=20)
ax3.view_init(elev=20, azim=45)

# 4. 자치구별 특성 분포 시각화
ax4 = fig.add_subplot(gs[1, 1])
district_stats = pd.DataFrame(X_scaled, columns=outlier_columns)
district_stats['자치구'] = district_info

# 자치구별 평균값 계산
district_means = district_stats.groupby('자치구').mean()

# 히트맵 시각화
sns.heatmap(district_means, 
            annot=True, 
            fmt='.2f', 
            cmap='YlOrRd',
            ax=ax4,
            cbar_kws={'label': '표준화된 값'})

ax4.set_title('자치구별 특성 분포', fontsize=15, pad=20)
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

# 이상치 제거 전후 데이터 수 비교
print(f"이상치 제거 전 데이터 수: {len(merged)}")
print(f"이상치 제거 후 데이터 수: {len(selected_df)}")
print(f"제거된 데이터 수: {len(merged) - len(selected_df)}")

# 각 자치구별 데이터 포인트 수 출력
print("\n자치구별 데이터 포인트 수:")
print(district_info.value_counts())

# PCA 설명력 출력
print("\n2차원 PCA 설명력:")
print(f"PC1: {pca2.explained_variance_ratio_[0]:.3f}")
print(f"PC2: {pca2.explained_variance_ratio_[1]:.3f}")
print(f"총 설명력: {sum(pca2.explained_variance_ratio_):.3f}")

print("\n3차원 PCA 설명력:")
print(f"PC1: {pca3.explained_variance_ratio_[0]:.3f}")
print(f"PC2: {pca3.explained_variance_ratio_[1]:.3f}")
print(f"PC3: {pca3.explained_variance_ratio_[2]:.3f}")
print(f"총 설명력: {sum(pca3.explained_variance_ratio_):.3f}")

# 자치구별 특성 분석
print("\n자치구별 특성 분석:")
print(district_means)

In [ ]:
# 필요한 라이브러리 import
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# 나눔고딕 폰트 설정
plt.rcParams['font.family'] = 'Nanum Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 스타일 설정
# plt.style.use('seaborn')
sns.set_palette("husl")

# 이상치 제거 함수
def remove_outliers(df, columns):
    df_clean = df.copy()
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    return df_clean

# 이상치가 많이 발생할 수 있는 변수들 선택
outlier_columns = ['총_유동인구_수', '청년층_유동인구_비율', '청년_매출_비율', '주말_매출_비율']

selected_df["자치구"] = merged["자치구"]

# 이상치 제거 전에 자치구 정보 저장
district_info = selected_df['자치구'].copy()

# 이상치 제거
selected_df = remove_outliers(selected_df, outlier_columns)

# 이상치 제거 후 남은 데이터에 대한 자치구 정보 업데이트
district_info = district_info[selected_df.index]

# 스케일링 및 PCA 적용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(selected_df[outlier_columns])

# 2차원 PCA
pca2 = PCA(n_components=2)
X_pca2 = pca2.fit_transform(X_scaled)

# 3차원 PCA
pca3 = PCA(n_components=3)
X_pca3 = pca3.fit_transform(X_scaled)

# K-means 클러스터링
n_clusters = 4  # 클러스터 수 설정
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

# 시각화를 위한 네 가지 방법 구현
fig = plt.figure(figsize=(20, 15))
gs = fig.add_gridspec(2, 2)

# 1. 밀도 기반 시각화 (KDE) - 개선된 버전
ax1 = fig.add_subplot(gs[0, 0])
kde = gaussian_kde(X_pca2.T)
x_min, x_max = X_pca2[:, 0].min(), X_pca2[:, 0].max()
y_min, y_max = X_pca2[:, 1].min(), X_pca2[:, 1].max()

x_margin = (x_max - x_min) * 0.1
y_margin = (y_max - y_min) * 0.1

x_grid = np.linspace(x_min - x_margin, x_max + x_margin, 200)
y_grid = np.linspace(y_min - y_margin, y_max + y_margin, 200)
X, Y = np.meshgrid(x_grid, y_grid)
positions = np.vstack([X.ravel(), Y.ravel()])

Z = np.reshape(kde(positions).T, X.shape)
ax1.contour(X, Y, Z, colors='black', alpha=0.5, linewidths=1)
ax1.contourf(X, Y, Z, cmap='viridis', alpha=0.3)

# 데이터 포인트 표시 (클러스터별 색상)
scatter1 = ax1.scatter(X_pca2[:, 0], X_pca2[:, 1], 
                      c=clusters, 
                      cmap='viridis',
                      alpha=0.6,
                      s=50,
                      edgecolor='black',
                      linewidth=0.5)

ax1.set_xlabel('PC1', fontsize=12)
ax1.set_ylabel('PC2', fontsize=12)
ax1.set_title('밀도 기반 시각화 (클러스터별)', fontsize=15, pad=20)
ax1.grid(True, linestyle='--', alpha=0.3)
plt.colorbar(scatter1, ax=ax1, label='클러스터')

# 2. 버블 차트 - 개선된 버전
ax2 = fig.add_subplot(gs[0, 1])
sizes = 100 + (np.random.rand(len(X_pca2)) * 200)  # 랜덤한 크기로 버블 생성

scatter2 = ax2.scatter(X_pca2[:, 0], X_pca2[:, 1],
                      c=clusters,
                      cmap='viridis',
                      s=sizes,
                      alpha=0.6,
                      edgecolor='black',
                      linewidth=0.5)

ax2.set_xlabel('PC1', fontsize=12)
ax2.set_ylabel('PC2', fontsize=12)
ax2.set_title('버블 차트 (클러스터별)', fontsize=15, pad=20)
ax2.grid(True, linestyle='--', alpha=0.3)
plt.colorbar(scatter2, ax=ax2, label='클러스터')

# 3. 3차원 PCA 시각화
ax3 = fig.add_subplot(gs[1, 0], projection='3d')
scatter3 = ax3.scatter(X_pca3[:, 0], X_pca3[:, 1], X_pca3[:, 2],
                      c=clusters,
                      cmap='viridis',
                      alpha=0.6,
                      s=50,
                      edgecolor='black',
                      linewidth=0.5)

ax3.set_xlabel('PC1', fontsize=12)
ax3.set_ylabel('PC2', fontsize=12)
ax3.set_zlabel('PC3', fontsize=12)
ax3.set_title('3차원 PCA 시각화 (클러스터별)', fontsize=15, pad=20)
plt.colorbar(scatter3, ax=ax3, label='클러스터')

# 그래프 회전을 위한 각도 설정
ax3.view_init(elev=20, azim=45)

# 4. 클러스터 중심점 시각화
ax4 = fig.add_subplot(gs[1, 1])
cluster_centers = kmeans.cluster_centers_
cluster_centers_pca = pca2.transform(cluster_centers)

# 데이터 포인트
ax4.scatter(X_pca2[:, 0], X_pca2[:, 1],
           c=clusters,
           cmap='viridis',
           alpha=0.3,
           s=50,
           edgecolor='black',
           linewidth=0.5)

# 클러스터 중심점
ax4.scatter(cluster_centers_pca[:, 0], cluster_centers_pca[:, 1],
           c='red',
           s=200,
           marker='*',
           edgecolor='black',
           linewidth=1,
           label='클러스터 중심')

ax4.set_xlabel('PC1', fontsize=12)
ax4.set_ylabel('PC2', fontsize=12)
ax4.set_title('클러스터 중심점 시각화', fontsize=15, pad=20)
ax4.grid(True, linestyle='--', alpha=0.3)
ax4.legend()

plt.tight_layout()
plt.show()

# 이상치 제거 전후 데이터 수 비교
print(f"이상치 제거 전 데이터 수: {len(merged)}")
print(f"이상치 제거 후 데이터 수: {len(selected_df)}")
print(f"제거된 데이터 수: {len(merged) - len(selected_df)}")

# 각 자치구별 데이터 포인트 수 출력
print("\n자치구별 데이터 포인트 수:")
print(district_info.value_counts())

# PCA 설명력 출력
print("\n2차원 PCA 설명력:")
print(f"PC1: {pca2.explained_variance_ratio_[0]:.3f}")
print(f"PC2: {pca2.explained_variance_ratio_[1]:.3f}")
print(f"총 설명력: {sum(pca2.explained_variance_ratio_):.3f}")

print("\n3차원 PCA 설명력:")
print(f"PC1: {pca3.explained_variance_ratio_[0]:.3f}")
print(f"PC2: {pca3.explained_variance_ratio_[1]:.3f}")
print(f"PC3: {pca3.explained_variance_ratio_[2]:.3f}")
print(f"총 설명력: {sum(pca3.explained_variance_ratio_):.3f}")

# 클러스터별 특성 분석
print("\n클러스터별 특성 분석:")
cluster_analysis = pd.DataFrame(X_scaled, columns=outlier_columns)
cluster_analysis['cluster'] = clusters
print("\n클러스터별 평균값:")
print(cluster_analysis.groupby('cluster').mean())

# 클러스터별 자치구 분포
print("\n클러스터별 자치구 분포:")
cluster_district = pd.DataFrame({
    '자치구': district_info,
    'cluster': clusters
})
print(cluster_district.groupby(['cluster', '자치구']).size().unstack(fill_value=0))